In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
variable85="rcp85_growing_degree_days_gt_4_monthly"
variable45="rcp45_growing_degree_days_gt_4_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-cl

In [5]:
files85

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp85_growing_degree_days_gt_4_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-cl

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 1812)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1950-01-01 ... 2100-12-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [34]:
# Seasonal example 30 y means
for s in dsEns45.time.dt.month:
    
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    
    # .... same as annual above

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-01-01 ... 2100-01-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-10-01 ... 2100-10-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-09-01 ... 2100-09-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-12-01 ... 2100-12-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-09-01 ... 2100-09-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

...

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-03-01 ... 2100-03-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-04-01 ... 2100-04-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-08-01 ... 2100-08-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-06-01 ... 2100-06-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-03-01 ... 2100-03-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
<xarray.Dataset>
Di

In [7]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:                       (lat: 320, lon: 416, realization: 11, time: 48)
Coordinates:
  * lat                           (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                           (lon) float32 -89.04521 ... -54.46326
  * time                          (time) datetime64[ns] 1981-01-01 ... 2098-07-01
  * realization                   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [8]:
#%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()
%time df45 = perc30yavg45.to_dataframe()
df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 21min 21s, sys: 2h 53min 22s, total: 3h 14min 43s
Wall time: 3min 2s


growing_degree_days_gt_4_p10  \
lat       lon        realization time                                       
66.623306 -89.045212 0           1981-01-01                     10.821252   
                                 1983-07-01                     15.402210   
                                 1986-01-01                     11.084667   
                                 1988-07-01                     18.399195   
                                 1991-01-01                     12.338125   
...                                                                   ...   
40.041039 -54.463261 10          2088-07-01                           NaN   
                                 2091-01-01                           NaN   
                                 2093-07-01                           NaN   
                                 2096-01-01                           NaN   
                                 2098-07-01                           NaN   

                                             growing_degree_days_gt_4_p50  \
lat       lon        realization time                                       
66.623306 -89.045212 0           1981-01-01                     14.862873   
                                 1983-07-01                     21.026163   
                                 1986-01-01                     14.554199   
                                 1988-07-01                     20.493355   
                                 1991-01-01                     14.577208   
...                                                                   ...   
40.041039 -54.463261 10          2088-07-01                           NaN   
                                 2091-01-01                           NaN   
                                 2093-07-01                           NaN   
                                 2096-01-01                           NaN   
                                 2098-07-01                           NaN   

                                             growing_degree_days_gt_4_p90  
lat       lon        realization time                                      
66.623306 -89.045212 0           1981-01-01                     16.372446  
                                 1983-07-01                     25.699488  
                                 1986-01-01                     17.871952  
                                 1988-07-01                     25.035944  
                                 1991-01-01                     22.286180  
...                                                                   ...  
40.041039 -54.463261 10          2088-07-01                           NaN  
                                 2091-01-01                           NaN  
                                 2093-07-01                           NaN  
                                 2096-01-01                           NaN  
                                 2098-07-01                           NaN  

[70287360 rows x 3 columns]

In [10]:
df452 = df45.dropna()
df453 = df452.reset_index()

In [22]:
df453.head()

,lat,lon,realization,time,growing_degree_days_gt_4_p10,growing_degree_days_gt_4_p50,growing_degree_days_gt_4_p90
0,66.623306,-89.045212,0,1981-01-01,10.821252,14.862873,16.372446
1,66.623306,-89.045212,0,1983-07-01,15.402210,21.026163,25.699488
2,66.623306,-89.045212,0,1986-01-01,11.084667,14.554199,17.871952
3,66.623306,-89.045212,0,1988-07-01,18.399195,20.493355,25.035944
4,66.623306,-89.045212,0,1991-01-01,12.338125,14.577208,22.286180


In [20]:
df454 = df453.loc[df453["realization"] == 0]


,lat,lon,realization,time,growing_degree_days_gt_4_p10,growing_degree_days_gt_4_p50,growing_degree_days_gt_4_p90
0,66.623306,-89.045212,0,1981-01-01,10.821252,14.862873,16.372446
1,66.623306,-89.045212,0,1983-07-01,15.402210,21.026163,25.699488
2,66.623306,-89.045212,0,1986-01-01,11.084667,14.554199,17.871952
3,66.623306,-89.045212,0,1988-07-01,18.399195,20.493355,25.035944
4,66.623306,-89.045212,0,1991-01-01,12.338125,14.577208,22.286180
...,...,...,...,...,...,...,...
40236283,40.041039,-74.045807,0,2088-07-01,323.439026,355.699768,368.675781
40236284,40.041039,-74.045807,0,2091-01-01,293.390106,333.323364,348.195770
40236285,40.041039,-74.045807,0,2093-07-01,328.889008,375.620667,387.173645
40236286,40.041039,-74.045807,0,2096-01-01,305.330688,325.372375,337.146088


In [33]:
a = df454.time.unique()
a.sort()
a

array(['1981-01-01T00:00:00.000000000', '1983-07-01T00:00:00.000000000',
       '1986-01-01T00:00:00.000000000', '1988-07-01T00:00:00.000000000',
       '1991-01-01T00:00:00.000000000', '1993-07-01T00:00:00.000000000',
       '1996-01-01T00:00:00.000000000', '1998-07-01T00:00:00.000000000',
       '2001-01-01T00:00:00.000000000', '2003-07-01T00:00:00.000000000',
       '2006-01-01T00:00:00.000000000', '2008-07-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2013-07-01T00:00:00.000000000',
       '2016-01-01T00:00:00.000000000', '2018-07-01T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000', '2023-07-01T00:00:00.000000000',
       '2026-01-01T00:00:00.000000000', '2028-07-01T00:00:00.000000000',
       '2031-01-01T00:00:00.000000000', '2033-07-01T00:00:00.000000000',
       '2036-01-01T00:00:00.000000000', '2038-07-01T00:00:00.000000000',
       '2041-01-01T00:00:00.000000000', '2043-07-01T00:00:00.000000000',
       '2046-01-01T00:00:00.000000000', '2048-07-01

In [27]:
df454.sort_values(by=["time"])


,lat,lon,realization,time,growing_degree_days_gt_4_p10,growing_degree_days_gt_4_p50,growing_degree_days_gt_4_p90
0,66.623306,-89.045212,0,1981-01-01,10.821252,14.862873,16.372446
30662544,46.624111,-72.212547,0,1981-01-01,145.192902,150.662704,156.452347
30663072,46.624111,-72.129219,0,1981-01-01,145.608353,151.063873,156.910126
5730384,60.706879,-74.212471,0,1981-01-01,11.476251,17.477814,21.421610
30663600,46.624111,-72.045891,0,1981-01-01,145.609955,151.022003,156.952469
...,...,...,...,...,...,...,...
26277023,48.874020,-65.296158,0,2098-07-01,138.756805,166.450180,185.955002
26277551,48.874020,-65.212830,0,2098-07-01,141.939148,169.579773,189.550705
26278079,48.874020,-65.129501,0,2098-07-01,142.680450,170.236084,189.923203
26288639,48.874020,-57.879791,0,2098-07-01,147.856400,170.564392,181.625473


## Round

In [ ]:
df45C = df45.copy()
df45C["growing_degree_days_gt_4_p10"] = round(df45C["growing_degree_days_gt_4_p10"],2)
df45C["growing_degree_days_gt_4_p50"] = round(df45C["growing_degree_days_gt_4_p50"],2)
df45C["growing_degree_days_gt_4_p90"] = round(df45C["growing_degree_days_gt_4_p90"],2)

df45C

## Eliminate 2011, Pivot Table and Create new columns 

In [ ]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [ ]:
dsEns85= ens.create_ensemble(files85)
dsEns85

In [ ]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

## Transform into DataFrame

In [ ]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

## Transform in Celsius and round

In [ ]:
df85C = df85.copy()
df85C["growing_degree_days_gt_4_p10"] = round(df85C["growing_degree_days_gt_4_p10"],2)
df85C["growing_degree_days_gt_4_p50"] = round(df85C["growing_degree_days_gt_4_p50"],2)
df85C["growing_degree_days_gt_4_p90"] = round(df85C["growing_degree_days_gt_4_p90"],2)

df85C

## Eliminate 2011, Pivot Table and Create new columns 

In [ ]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

# Merge df45 and 85

In [ ]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

# Extract hist

In [ ]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [ ]:
dftg1

# Merge both hist_50 and get the mean

In [ ]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

In [ ]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

# Merge means with the 45 and 85 DF

In [ ]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [ ]:
dftg_all

In [ ]:
dftg_all.to_feather("growing_degree_days_gt_4_p10_annual.feather")

In [ ]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/growing_degree_days_gt_4_p10_annual.feather")

In [ ]:
dftg_all 

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [ ]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+replace_all(gdf[name].iloc[0], d)+"_DJC_p10_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_DJC_annual.json", driver="GeoJSON")